Tenemos varios datasets con el número de turistas según nacionalidad de origen y otro con el gasto realizado según nacionalidad de origen. 

Queremos unirlo en uno solo:

- Filtraremos por las Comunidades Autónomas que nos interesen.
- Deberemos eliminar los paises de los que no tenemos datos acerca del gasto.
- El gasto se indica por trimestre y la llegada de turistas por mes.

In [30]:
import pandas as pd

In [31]:
#Cargamos los csv de llegadas y los concatenamos

csv_2020 = pd.read_csv("data/llegadas_segun_nacionalidad_2020_movil.csv", delimiter= ";", encoding = 'Latin1')
csv_2021 = pd.read_csv("data/llegadas_segun_nacionalidad_2021_movil.csv", delimiter= ";", encoding = 'Latin1')
csv_2022 = pd.read_csv("data/llegadas_segun_nacionalidad_2022_movil.csv", delimiter= ";", encoding = 'Latin1')
csv_2023 = pd.read_csv("data/llegadas_segun_nacionalidad_2023_movil.csv", delimiter= ";", encoding = 'Latin1')


In [32]:
df_llegadas = pd.concat([csv_2023, csv_2022, csv_2021, csv_2020], ignore_index= True)
df_llegadas.head()

,RESIDENCIA/ORIGEN,Continentes,Países,Concepto turístico,CCAA de destino,Periodo,Total
0,Total,Europa,NaN,Turistas,"Balears, Illes",2023M09,1.874.324
1,Total,Europa,NaN,Turistas,"Balears, Illes",2023M08,2.091.801
2,Total,Europa,NaN,Turistas,"Balears, Illes",2023M07,2.136.840
3,Total,Europa,NaN,Turistas,"Balears, Illes",2023M06,1.887.011
4,Total,Europa,NaN,Turistas,"Balears, Illes",2023M05,1.588.234


In [33]:
#eliminamos el último trimestre del que no tenemos datos en df_gasto
df_llegadas.drop(df_llegadas[df_llegadas["Periodo"].isin(["2023M09","2023M08","2023M07"])].index, inplace = True)

In [34]:
#Cargamos el csv de gasto 
#lo filtramos por las comunidades de Baleares y Canarias

df_gasto = pd.read_csv("data/gasto_segun_nacionalidad_turista.csv", delimiter= ";")
df_gasto = df_gasto[df_gasto["Comunidades y Ciudades Autónomas de destino"].isin(['Balears, Illes','Canarias'])]

In [35]:
df_gasto.head()

,Comunidades y Ciudades Autónomas de destino,País de residencia,Periodo,Total
2310,"Balears, Illes",Total,2023T2,2.501.458.677
2311,"Balears, Illes",Total,2023T1,"458.328.072,63"
2312,"Balears, Illes",Total,2022T4,"936.363.387,87"
2313,"Balears, Illes",Total,2022T3,"2.936.909.174,3"
2314,"Balears, Illes",Total,2022T2,2.534.873.858


In [36]:
#eliminamos las filas de los años que no tenemos en df_llegadas

df_gasto = df_gasto[df_gasto["Periodo"].str.contains("2020|2021|2022|2023", regex=True) ]

In [37]:
#Descartamos de csv_gasto los paises que no nos interesan

df_gasto.drop(df_gasto[df_gasto["País de residencia"].isin(["Total","Otros Países Europeos","Resto de América","Resto del mundo"])].index, inplace = True)    

In [38]:
#hacemos una lista de los paises restantes para filtrar los csv de llegadas

lista_paises = []
for i in df_gasto['País de residencia'].unique():
    lista_paises.append(i)

In [39]:
#quitamos de df_llegadas los paises que no tenemos en df_gasto

df_llegadas= df_llegadas[df_llegadas["Países"].isin(lista_paises)]

In [40]:
#Convertimos el datos de Total en df_gasto y en df_llegadas a float. 

df_gasto["Total"] = df_gasto["Total"].str.replace('\.$', '0', regex= True).str.replace('\.', '', regex= True).str.replace(',', '.', regex= True).astype(dtype="float64")
df_llegadas["Total"] = df_llegadas["Total"].str.replace('\.$', '0', regex= True).str.replace('\.', '', regex= True).str.replace(',', '.', regex= True).astype("float64")


In [41]:
#separamos en dos dataframe los dos tipos de conceptos turisticos. Reseteamos los índices
#de momento sólo usaremos el de Turistas

df_turistas = df_llegadas[df_llegadas["Concepto turístico"] == "Turistas"].reset_index(drop = True)
df_dias = df_llegadas[df_llegadas["Concepto turístico"] == "Duración media de los viajes"].drop(['RESIDENCIA/ORIGEN', 'Continentes', 'Concepto turístico' ], axis=1).reset_index(drop = True)

In [42]:
#creamos una funcion para crear la columna de trimestre

def trimestre(dato):
    if dato[5:7] == "01" or dato[5:7] == "02" or dato[5:7] == "03":
        return f"{dato[0:4]}T1" 
    elif dato[5:7] == "04" or dato[5:7] == "05" or dato[5:7] == "06":
        return f"{dato[0:4]}T2" 
    elif dato[5:7] == "07" or dato[5:7] == "08" or dato[5:7] == "09":
        return f"{dato[0:4]}T3" 
    else:
        return f"{dato[0:4]}T4"

In [43]:
#creamos la columna en df_turistas

df_turistas["Trimestre"] = df_turistas["Periodo"].apply(trimestre)

In [44]:
#Sumamos los totales por trimestre en df_turistas

df_turistas_gb = df_turistas.groupby(by=["Trimestre", "CCAA de destino", "Países"]).sum().reset_index()

In [45]:
#Creamos la columna trimestre en el df_dias
df_dias["Trimestre"] = df_dias["Periodo"].apply(trimestre)

In [46]:
#Sacamos la media de estancia por trimestre

df_dias_gb = df_dias.groupby(by=["Trimestre", "CCAA de destino", "Países"]).mean().round(2).reset_index()

In [47]:
#unimos estos dos últimos dataframes en uno solo

df_llegadas_final = df_turistas_gb.merge(df_dias_gb, on=["Trimestre", "CCAA de destino", "Países"])

In [48]:
#Cambiamos los nombres de las columnas de total
nombres = {"Total_x": "Total turistas", "Total_y": "Dias estancia media"}
df_llegadas_final.rename(columns = nombres, inplace= True)

In [49]:
#Cambiamos los nombres de las columna en df_gasto
nombres_2= {"Comunidades y Ciudades Autónomas de destino":"CCAA de destino", 
            "País de residencia":"Países",
            "Periodo":"Trimestre",
            "Total": "Total gasto"}
df_gasto.rename(columns = nombres_2, inplace= True)

In [50]:
#unimos los datos de llegada con los de gastos

df_llegadas_gasto = df_gasto.merge(df_llegadas_final, on=["Trimestre", "CCAA de destino", "Países"])

In [51]:
df_llegadas_gasto

,CCAA de destino,Países,Trimestre,Total gasto,Total turistas,Dias estancia media
0,"Balears, Illes",Alemania,2023T2,6.431207e+08,1560273.0,6.40
1,"Balears, Illes",Alemania,2023T1,1.910132e+08,340599.0,7.50
2,"Balears, Illes",Alemania,2022T4,2.983763e+08,622127.0,7.87
3,"Balears, Illes",Alemania,2022T3,5.497259e+08,1733548.0,7.77
4,"Balears, Illes",Alemania,2022T2,7.041295e+08,1618693.0,6.80
...,...,...,...,...,...,...
863,Canarias,Turquía,2021T1,5.348635e+04,265.0,5.57
864,Canarias,Turquía,2020T4,7.354406e+04,316.0,4.87
865,Canarias,Turquía,2020T3,1.140196e+05,200.0,5.20
866,Canarias,Turquía,2020T2,0.000000e+00,231.0,4.97


In [52]:
#Guardamos el dataframe para posterior tratamiento con PowerBI
#El formato float es con coma y dos decimales para que lo podamos cargar correctamente

df_llegadas_gasto.to_csv('data/gasto_llegadas_segun_nacionalidad.csv', index= False, decimal = ",", float_format= "%.2f")